In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# vanilla RSA adapted from seminar's RSA model

## note to self: barebones for now

class RSAModel()
  def __init(self, objects, messages, truth_matrix, alpha, priors, cost_function)
    if truth_matrix.shape != (len(objects, len(messages))):
      raise ValueError("""Truth matrix must be m x n, 
      where m is number of objects and n is the number of messages""")
    self.objects = objects
    self.messages = messages
    self.truth_matrix = truth
    self.alpha = alpha
    self.priors = priors
    self.cost_function = cost_function

  def __normalize(self, arr):
    return arr / arr.sum(axis=1)


